# Визуализация взрыва градиентов (Exploding Gradients)

В этом эксперименте мы увидим, как неправильная инициализация весов в глубокой сети приводит к тому, что значения активаций и градиентов растут экспоненциально, делая обучение невозможным.

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)

set_seed()

## 1. Создание глубокой линейной сети

Мы создадим сеть из 50 последовательных линейных слоев. Мы намеренно инициализируем их веса так, чтобы они были чуть больше единицы ($W \approx 1.5 \cdot I$).

In [ ]:
L = 50 # Количество слоев
D = 128 # Размерность

layers = []
for i in range(L):
    layer = nn.Linear(D, D, bias=False)
    # Намеренная ПЛОХАЯ инициализация: веса чуть больше 1
    # Это гарантирует взрыв при умножении матриц друг на друга
    with torch.no_grad():
        layer.weight.copy_(torch.eye(D) * 1.5 + torch.randn(D, D) * 0.01)
    layers.append(layer)

model = nn.Sequential(*layers)
# Визуализация структуры параметров
for name, param in model.named_parameters():
    print(f"{name:40} | Shape: {str(list(param.shape)):20} | Mean: {param.mean().item():.4f}")

## 2. Forward Pass: Рост активаций

Посмотрим, как меняется норма вектора данных по мере прохождения сквозь 50 слоев.

In [ ]:
x = torch.randn(1, D)
activations_norms = []

curr_x = x
for layer in model:
    curr_x = layer(curr_x)
    activations_norms.append(curr_x.norm().item())

plt.figure(figsize=(10, 4))
plt.plot(activations_norms, 'r-o')
plt.yscale('log')
plt.title("Норма активаций (Forward Pass) - Логарифмическая шкала")
plt.xlabel("Слой")
plt.ylabel("L2 Norm")
plt.grid(True)
plt.show()

print(f"Начальная норма: {activations_norms[0]:.2f}")
print(f"Финальная норма через 50 слоев: {activations_norms[-1]:.2e}")

## 3. Backward Pass: Взрыв градиентов

Теперь мы вычислим лосс и сделаем `backward()`. Мы будем записывать норму градиента для каждого слоя.

In [ ]:
# 1. Считаем градиенты
loss = curr_x.sum()
model.zero_grad()
loss.backward()

gradient_norms = []
layer_names = []
for i, layer in enumerate(model):
    norm = layer.weight.grad.norm().item()
    gradient_norms.append(norm)
    layer_names.append(f'Layer {i}')

# 2. Визуализация взрыва
plt.figure(figsize=(12, 6))

# Левый график: Линейная шкала (тут будет виден сам 'взрыв')
plt.subplot(1, 2, 1)
plt.plot(gradient_norms, 'b-')
plt.title("Взрыв градиентов (Линейная шкала)")
plt.xlabel("Номер слоя (0-вход, 50-выход)")
plt.ylabel("Frobenius Norm of Gradient")
plt.grid(True)

# Правый график: Логарифмическая шкала (тут видна динамика)
plt.subplot(1, 2, 2)
plt.plot(gradient_norms, 'g-')
plt.yscale('log')
plt.title("Взрыв градиентов (Log-шкала)")
plt.xlabel("Номер слоя")
plt.grid(True)

plt.tight_layout()
plt.show()

print(f'Градиент последнего слоя (Layer 49): {gradient_norms[-1]:.2f}')
print(f'Градиент первого слоя (Layer 0):    {gradient_norms[0]:.2e}')
print('\nОбрати внимание: градиент растет ПРИ ОБРАТНОМ ХОДЕ (от конца к началу),')
print('поэтому на Layer 0 он в миллионы раз больше, чем на Layer 49!')

## 4. Как это исправляет LayerNorm?

Если мы добавим LayerNorm между нашими слоями, он будет принудительно 'сжимать' активации обратно к нормальному распределению на каждом шаге.